In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import multiprocessing
from scipy.io import loadmat
from spectral import *
from functions import *
import time

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
def task(r):
    np.random.seed(r)
    Y = np.zeros([m,n])
    for i in range(p):
        Y[i,:] = 1*np.random.randn(n) + i
    for i in range(q):
        Y[:,i] = 1*np.random.randn(m) + i

    for i in range(p,m):
        for j in range(q,n):
            Y[i,j] = np.sum(Y[(i-p):(i+1),(j-q):(j+1)] * beta) + 0.1 * np.sum(np.random.randn(1))

    cov_loc = causal_get_cov_loc(order)

    XX = np.zeros([(m-p), (n-q),len(cov_loc)])
    i = 0
    for loc in cov_loc:
        k = loc[0]
        l = loc[1]
        XX[:,:,i] = Y[(p+k):(m+k), (q+l):(n+l)]
        i = i + 1

    start_lev = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        lev_temp = np.zeros([m-p,n-q])
        matrix_inv = np.linalg.inv(tensor_multiply(XX_temp, XX_temp))
        for i in range(m-p):
            for j in range(n-q):
                lev_temp[i,j] = XX_temp[i,j,:].T @ matrix_inv @ XX_temp[i,j,:]
        phi_lev = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:], lev_score = lev_temp, subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_lev)
    end_lev = time.time()
    time_lev = end_lev - start_lev
    MSE_lev = np.min(MSE_temp)
    MSE_para_lev = np.mean((phi - phi_lev)**2)

    start_fast = time.time()
    leverage_score = causal_fast_2d_AR_leverage_score(Y, order=order, subsample_size = subsample_size)
    phi_fast = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = leverage_score[p:,q:], subsample_size = subsample_size)
    end_fast = time.time()
    time_fast = end_fast - start_fast
    MSE_fast = MSE_calculate(XX, Y[p:,q:], phi_fast)
    MSE_para_fast = np.mean((phi - phi_fast)**2)

    start_OLS = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_OLS = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:]).reshape(-1)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_OLS)
    end_OLS = time.time()
    time_OLS = end_OLS - start_OLS
    MSE_OLS = np.min(MSE_temp)
    MSE_para_OLS = np.mean((phi - phi_OLS)**2)

    start_random = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_random = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = np.ones([m-p,n-q]), subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_random)
    end_random = time.time()
    time_random = end_random - start_random
    MSE_random = np.min(MSE_temp)
    MSE_para_random = np.mean((phi - phi_random)**2)

    return MSE_lev, MSE_fast, MSE_OLS, MSE_random, time_lev, time_fast, time_OLS, time_random, MSE_para_lev, MSE_para_fast, MSE_para_OLS, MSE_para_random

In [ ]:
order_list = [[2,2], [3,3], [4,4], [5,5], [5,6], [6,6], [5,8], [7,7]]
nrep = 20
SEED = 2024
subsample_size = 10000

MSE_lev_list = np.zeros(len(order_list))
MSE_fast_list = np.zeros(len(order_list))
MSE_OLS_list = np.zeros(len(order_list))
MSE_random_list = np.zeros(len(order_list))
MSE_para_lev_list = np.zeros(len(order_list))
MSE_para_fast_list = np.zeros(len(order_list))
MSE_para_OLS_list = np.zeros(len(order_list))
MSE_para_random_list = np.zeros(len(order_list))
time_lev_list = np.zeros(len(order_list))
time_fast_list = np.zeros(len(order_list))
time_OLS_list = np.zeros(len(order_list))
time_random_list = np.zeros(len(order_list))
time_XX_list = np.zeros(len(order_list))

MSE_lev_std_list = np.zeros(len(order_list))
MSE_fast_std_list = np.zeros(len(order_list))
MSE_OLS_std_list = np.zeros(len(order_list))
MSE_random_std_list = np.zeros(len(order_list))
MSE_para_lev_std_list = np.zeros(len(order_list))
MSE_para_fast_std_list = np.zeros(len(order_list))
MSE_para_OLS_std_list = np.zeros(len(order_list))
MSE_para_random_std_list = np.zeros(len(order_list))
time_lev_std_list = np.zeros(len(order_list))
time_fast_std_list = np.zeros(len(order_list))
time_OLS_std_list = np.zeros(len(order_list))
time_random_std_list = np.zeros(len(order_list))

MSE_lev = np.zeros(nrep)
MSE_fast = np.zeros(nrep)
MSE_OLS = np.zeros(nrep)
MSE_random = np.zeros(nrep)
time_lev = np.zeros(nrep)
time_fast = np.zeros(nrep)
time_fast_trick = np.zeros(nrep)
time_OLS = np.zeros(nrep)
time_random = np.zeros(nrep)
time_XX = np.zeros(nrep)

m = 1000
n = 1000

order_num = 0
para_num_list = []

for order in order_list:
    p = order[0]
    q = order[1]
    beta = beta_generate(order, method = '3')
    phi = beta.reshape(-1)[0:(p*q+p+q)]
    para_num = (p+1)*(q+1)-1
    para_num_list.append(para_num)
    print(order)
    with multiprocessing.Pool(processes = 20) as pool:
        results = pool.map(task, range(nrep))
        A = np.array(results)
        MSE_lev = A[:,0]
        MSE_fast = A[:,1]
        MSE_OLS = A[:,2]
        MSE_random = A[:,3]
        time_lev = A[:,4]
        time_fast = A[:,5]
        time_OLS = A[:,6]
        time_random = A[:,7]
        MSE_para_lev = A[:,8]
        MSE_para_fast = A[:,9]
        MSE_para_OLS = A[:,10]
        MSE_para_random = A[:,11]

    MSE_lev_list[order_num] = np.mean(MSE_lev)
    MSE_fast_list[order_num] = np.mean(MSE_fast)
    MSE_OLS_list[order_num] = np.mean(MSE_OLS)
    MSE_random_list[order_num] = np.mean(MSE_random)
    MSE_para_lev_list[order_num] = np.mean(MSE_para_lev)
    MSE_para_fast_list[order_num] = np.mean(MSE_para_fast)
    MSE_para_OLS_list[order_num] = np.mean(MSE_para_OLS)
    MSE_para_random_list[order_num] = np.mean(MSE_para_random)

    MSE_lev_std_list[order_num] = np.std(MSE_lev)
    MSE_fast_std_list[order_num] = np.std(MSE_fast)
    MSE_OLS_std_list[order_num] = np.std(MSE_OLS)
    MSE_random_std_list[order_num] = np.std(MSE_random)
    MSE_para_lev_std_list[order_num] = np.std(MSE_para_lev)
    MSE_para_fast_std_list[order_num] = np.std(MSE_para_fast)
    MSE_para_OLS_std_list[order_num] = np.std(MSE_para_OLS)
    MSE_para_random_std_list[order_num] = np.std(MSE_para_random)

    time_lev_list[order_num] = np.mean(time_lev)
    time_fast_list[order_num] = np.mean(time_fast)
    time_OLS_list[order_num] = np.mean(time_OLS)
    time_random_list[order_num] = np.mean(time_random)

    time_lev_std_list[order_num] = np.std(time_lev)
    time_fast_std_list[order_num] = np.std(time_fast)
    time_OLS_std_list[order_num] = np.std(time_OLS)
    time_random_std_list[order_num] = np.std(time_random)

    order_num = order_num + 1

In [ ]:
plt.errorbar(para_num_list, MSE_para_lev_list, yerr = MSE_para_lev_std_list, color = 'orange', fmt = 'o-', label = 'LEV-exact')
plt.errorbar(para_num_list, MSE_para_fast_list, yerr = MSE_para_fast_std_list, color = 'red', fmt = '^-', label = 'LEV-appr')
plt.errorbar(para_num_list, MSE_para_OLS_list, yerr = MSE_para_OLS_std_list, color = 'black', fmt = 's-', label = 'OLS')
plt.errorbar(para_num_list, MSE_para_random_list, yerr = MSE_para_random_std_list, color = 'blue', fmt = '.-', label = 'UNIF')
plt.yscale('log')
plt.xlabel('number of parameter d', fontsize = 20)
plt.ylabel('MSE', fontsize = 20)
plt.tick_params(axis='both', labelsize=14)
plt.legend(fontsize = 12)

In [7]:
def task(r):
    np.random.seed(r)
    Y = np.zeros([m,n])
    for i in range(p):
        Y[i,:] = 1*np.random.randn(n) + i
    for i in range(q):
        Y[:,i] = 1*np.random.randn(m) + i

    for i in range(p,m):
        for j in range(q,n):
            Y[i,j] = np.sum(Y[(i-p):(i+1),(j-q):(j+1)] * beta) + 0.1 * np.sum(np.random.randn(1))

    cov_loc = causal_get_cov_loc(order)

    XX = np.zeros([(m-p), (n-q),len(cov_loc)])
    i = 0
    for loc in cov_loc:
        k = loc[0]
        l = loc[1]
        XX[:,:,i] = Y[(p+k):(m+k), (q+l):(n+l)]
        i = i + 1

    start_lev = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        lev_temp = np.zeros([m-p,n-q])
        matrix_inv = np.linalg.inv(tensor_multiply(XX_temp, XX_temp))
        for i in range(m-p):
            for j in range(n-q):
                lev_temp[i,j] = XX_temp[i,j,:].T @ matrix_inv @ XX_temp[i,j,:]
        phi_lev = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:], lev_score = lev_temp, subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_lev)
    end_lev = time.time()
    time_lev = end_lev - start_lev
    MSE_lev = np.min(MSE_temp)
    MSE_para_lev = np.mean((phi - phi_lev)**2)

    start_fast = time.time()
    leverage_score = causal_fast_2d_AR_leverage_score(Y, order=order, subsample_size = subsample_size)
    phi_fast = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = leverage_score[p:,q:], subsample_size = subsample_size)
    end_fast = time.time()
    time_fast = end_fast - start_fast
    MSE_fast = MSE_calculate(XX, Y[p:,q:], phi_fast)
    MSE_para_fast = np.mean((phi - phi_fast)**2)

    start_OLS = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_OLS = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:]).reshape(-1)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_OLS)
    end_OLS = time.time()
    time_OLS = end_OLS - start_OLS
    MSE_OLS = np.min(MSE_temp)
    MSE_para_OLS = np.mean((phi - phi_OLS)**2)

    start_random = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_random = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = np.ones([m-p,n-q]), subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_random)
    end_random = time.time()
    time_random = end_random - start_random
    MSE_random = np.min(MSE_temp)
    MSE_para_random = np.mean((phi - phi_random)**2)

    return MSE_lev, MSE_fast, MSE_OLS, MSE_random, time_lev, time_fast, time_OLS, time_random, MSE_para_lev, MSE_para_fast, MSE_para_OLS, MSE_para_random

In [ ]:
size_list = [[150,150], [150,300], [300,300],[300,500], [500,1000], [1000,1000], [1000,3000]]
nrep = 20
SEED = 2024
subsample_size = 10000

MSE_lev_list = np.zeros(len(size_list))
MSE_fast_list = np.zeros(len(size_list))
MSE_OLS_list = np.zeros(len(size_list))
MSE_random_list = np.zeros(len(size_list))
MSE_para_lev_list = np.zeros(len(size_list))
MSE_para_fast_list = np.zeros(len(size_list))
MSE_para_OLS_list = np.zeros(len(size_list))
MSE_para_random_list = np.zeros(len(size_list))
time_lev_list = np.zeros(len(size_list))
time_fast_list = np.zeros(len(size_list))
time_OLS_list = np.zeros(len(size_list))
time_random_list = np.zeros(len(size_list))
time_XX_list = np.zeros(len(size_list))

MSE_lev_std_list = np.zeros(len(size_list))
MSE_fast_std_list = np.zeros(len(size_list))
MSE_OLS_std_list = np.zeros(len(size_list))
MSE_random_std_list = np.zeros(len(size_list))
MSE_para_lev_std_list = np.zeros(len(size_list))
MSE_para_fast_std_list = np.zeros(len(size_list))
MSE_para_OLS_std_list = np.zeros(len(size_list))
MSE_para_random_std_list = np.zeros(len(size_list))
time_lev_std_list = np.zeros(len(size_list))
time_fast_std_list = np.zeros(len(size_list))
time_OLS_std_list = np.zeros(len(size_list))
time_random_std_list = np.zeros(len(size_list))

MSE_lev = np.zeros(nrep)
MSE_fast = np.zeros(nrep)
MSE_OLS = np.zeros(nrep)
MSE_random = np.zeros(nrep)
time_lev = np.zeros(nrep)
time_fast = np.zeros(nrep)
time_fast_trick = np.zeros(nrep)
time_OLS = np.zeros(nrep)
time_random = np.zeros(nrep)
time_XX = np.zeros(nrep)

order = [5,5]
p = order[0]
q = order[1]
beta = beta_generate(order, method = '3')
phi = beta.reshape(-1)[0:(p*q+p+q)]
size_num = 0

size_total_list = []

for [m,n] in size_list:
    print([m,n])
    size_total_list.append(m*n)
    with multiprocessing.Pool(processes = 20) as pool:
        results = pool.map(task, range(nrep))
        A = np.array(results)
        MSE_lev = A[:,0]
        MSE_fast = A[:,1]
        MSE_OLS = A[:,2]
        MSE_random = A[:,3]
        time_lev = A[:,4]
        time_fast = A[:,5]
        time_OLS = A[:,6]
        time_random = A[:,7]
        MSE_para_lev = A[:,8]
        MSE_para_fast = A[:,9]
        MSE_para_OLS = A[:,10]
        MSE_para_random = A[:,11]

    MSE_lev_list[size_num] = np.mean(MSE_lev)
    MSE_fast_list[size_num] = np.mean(MSE_fast)
    MSE_OLS_list[size_num] = np.mean(MSE_OLS)
    MSE_random_list[size_num] = np.mean(MSE_random)
    MSE_para_lev_list[size_num] = np.mean(MSE_para_lev)
    MSE_para_fast_list[size_num] = np.mean(MSE_para_fast)
    MSE_para_OLS_list[size_num] = np.mean(MSE_para_OLS)
    MSE_para_random_list[size_num] = np.mean(MSE_para_random)

    MSE_lev_std_list[size_num] = np.std(MSE_lev)
    MSE_fast_std_list[size_num] = np.std(MSE_fast)
    MSE_OLS_std_list[size_num] = np.std(MSE_OLS)
    MSE_random_std_list[size_num] = np.std(MSE_random)
    MSE_para_lev_std_list[size_num] = np.std(MSE_para_lev)
    MSE_para_fast_std_list[size_num] = np.std(MSE_para_fast)
    MSE_para_OLS_std_list[size_num] = np.std(MSE_para_OLS)
    MSE_para_random_std_list[size_num] = np.std(MSE_para_random)

    time_lev_list[size_num] = np.mean(time_lev)
    time_fast_list[size_num] = np.mean(time_fast)
    time_OLS_list[size_num] = np.mean(time_OLS)
    time_random_list[size_num] = np.mean(time_random)

    time_lev_std_list[size_num] = np.std(time_lev)
    time_fast_std_list[size_num] = np.std(time_fast)
    time_OLS_std_list[size_num] = np.std(time_OLS)
    time_random_std_list[size_num] = np.std(time_random)

    size_num = size_num + 1

In [ ]:
plt.errorbar(size_total_list, MSE_para_lev_list, yerr = MSE_para_lev_std_list, color = 'orange', fmt = 'o-', label = 'LEV-exact')
plt.errorbar(size_total_list, MSE_para_fast_list, yerr = MSE_para_fast_std_list, color = 'red', fmt = '^-', label = 'LEV-appr')
plt.errorbar(size_total_list, MSE_para_OLS_list, yerr = MSE_para_OLS_std_list, color = 'black', fmt = 's-', label = 'OLS')
plt.errorbar(size_total_list, MSE_para_random_list, yerr = MSE_para_random_std_list, color = 'blue', fmt = '.-', label = 'UNIF')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('size of time series N', fontsize = 20)
plt.ylabel('MSE', fontsize = 20)
plt.tick_params(axis='both', labelsize=14)
plt.xticks(ticks=[3e4, 1e5, 3e5, 1e6, 3e6], labels=[rf'$3\times 10^4$', '$10^5$', rf'$3\times 10^5$', '$10^6$', rf'$3\times 10^6$'])
#plt.xlim([1e4, 1e6])
plt.ylim([1e-7,1e-4])
plt.legend(fontsize = 12)

In [2]:
def task(r):
    np.random.seed(r)
    Y = np.zeros([m,n])
    for i in range(p):
        Y[i,:] = 1*np.random.randn(n) + i
    for i in range(q):
        Y[:,i] = 1*np.random.randn(m) + i

    for i in range(p,m):
        for j in range(q,n):
            Y[i,j] = np.sum(Y[(i-p):(i+1),(j-q):(j+1)] * beta) + 0.1 * np.sum(np.random.randn(1))

    cov_loc = causal_get_cov_loc(order)

    XX = np.zeros([(m-p), (n-q),len(cov_loc)])
    i = 0
    for loc in cov_loc:
        k = loc[0]
        l = loc[1]
        XX[:,:,i] = Y[(p+k):(m+k), (q+l):(n+l)]
        i = i + 1

    start_lev = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        lev_temp = np.zeros([m-p,n-q])
        matrix_inv = np.linalg.inv(tensor_multiply(XX_temp, XX_temp))
        for i in range(m-p):
            for j in range(n-q):
                lev_temp[i,j] = XX_temp[i,j,:].T @ matrix_inv @ XX_temp[i,j,:]
        phi_lev = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:], lev_score = lev_temp, subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_lev)
    end_lev = time.time()
    time_lev = end_lev - start_lev
    MSE_lev = np.min(MSE_temp)
    MSE_para_lev = np.mean((phi - phi_lev)**2)

    start_fast = time.time()
    leverage_score = causal_fast_2d_AR_leverage_score(Y, order=order, subsample_size = subsample_size)
    phi_fast = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = leverage_score[p:,q:], subsample_size = subsample_size)
    end_fast = time.time()
    time_fast = end_fast - start_fast
    MSE_fast = MSE_calculate(XX, Y[p:,q:], phi_fast)
    MSE_para_fast = np.mean((phi - phi_fast)**2)

    start_OLS = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_OLS = two_dim_param_estimate_leverage(XX_temp, Y[p:,q:]).reshape(-1)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_OLS)
    end_OLS = time.time()
    time_OLS = end_OLS - start_OLS
    MSE_OLS = np.min(MSE_temp)
    MSE_para_OLS = np.mean((phi - phi_OLS)**2)

    start_random = time.time()
    MSE_temp = np.ones(len(cov_loc))
    for l in range(len(cov_loc)):
        XX_temp = XX[:,:,:(l+1)]
        phi_random = two_dim_param_estimate_leverage(XX, Y[p:,q:], lev_score = np.ones([m-p,n-q]), subsample_size = subsample_size)
        MSE_temp[l] = MSE_calculate(XX_temp, Y[p:,q:], phi_random)
    end_random = time.time()
    time_random = end_random - start_random
    MSE_random = np.min(MSE_temp)
    MSE_para_random = np.mean((phi - phi_random)**2)

    return MSE_lev, MSE_fast, MSE_OLS, MSE_random, time_lev, time_fast, time_OLS, time_random, MSE_para_lev, MSE_para_fast, MSE_para_OLS, MSE_para_random

In [ ]:
subsample_size_list = [100, 300, 1000, 3000, 10000]
nrep = 20
SEED = 2024
#np.random.seed(SEED)

MSE_lev_list = np.zeros(len(subsample_size_list))
MSE_fast_list = np.zeros(len(subsample_size_list))
MSE_OLS_list = np.zeros(len(subsample_size_list))
MSE_random_list = np.zeros(len(subsample_size_list))
MSE_para_lev_list = np.zeros(len(subsample_size_list))
MSE_para_fast_list = np.zeros(len(subsample_size_list))
MSE_para_OLS_list = np.zeros(len(subsample_size_list))
MSE_para_random_list = np.zeros(len(subsample_size_list))
time_lev_list = np.zeros(len(subsample_size_list))
time_fast_list = np.zeros(len(subsample_size_list))
time_OLS_list = np.zeros(len(subsample_size_list))
time_random_list = np.zeros(len(subsample_size_list))

MSE_lev_std_list = np.zeros(len(subsample_size_list))
MSE_fast_std_list = np.zeros(len(subsample_size_list))
MSE_OLS_std_list = np.zeros(len(subsample_size_list))
MSE_random_std_list = np.zeros(len(subsample_size_list))
MSE_para_lev_std_list = np.zeros(len(subsample_size_list))
MSE_para_fast_std_list = np.zeros(len(subsample_size_list))
MSE_para_OLS_std_list = np.zeros(len(subsample_size_list))
MSE_para_random_std_list = np.zeros(len(subsample_size_list))
time_lev_std_list = np.zeros(len(subsample_size_list))
time_fast_std_list = np.zeros(len(subsample_size_list))
time_OLS_std_list = np.zeros(len(subsample_size_list))
time_random_std_list = np.zeros(len(subsample_size_list))

MSE_lev = np.zeros(nrep)
MSE_fast = np.zeros(nrep)
MSE_OLS = np.zeros(nrep)
MSE_random = np.zeros(nrep)
time_lev = np.zeros(nrep)
time_fast = np.zeros(nrep)
time_fast_trick = np.zeros(nrep)
time_OLS = np.zeros(nrep)
time_random = np.zeros(nrep)
time_XX = np.zeros(nrep)

m = 1000
n = 1000
Y = np.zeros([m,n])
p = 5
q = 5
order = [p,q]
beta = beta_generate(order, method = '3')
phi = beta.reshape(-1)[0:(p*q+p+q)]

size_num = 0
for subsample_size in subsample_size_list:
    print(subsample_size)
    with multiprocessing.Pool(processes = 20) as pool:
        results = pool.map(task, range(nrep))
        A = np.array(results)
        MSE_lev = A[:,0]
        MSE_fast = A[:,1]
        MSE_OLS = A[:,2]
        MSE_random = A[:,3]
        time_lev = A[:,4]
        time_fast = A[:,5]
        time_OLS = A[:,6]
        time_random = A[:,7]
        MSE_para_lev = A[:,8]
        MSE_para_fast = A[:,9]
        MSE_para_OLS = A[:,10]
        MSE_para_random = A[:,11]

    MSE_lev_list[size_num] = np.mean(MSE_lev)
    MSE_fast_list[size_num] = np.mean(MSE_fast)
    MSE_OLS_list[size_num] = np.mean(MSE_OLS)
    MSE_random_list[size_num] = np.mean(MSE_random)
    MSE_para_lev_list[size_num] = np.mean(MSE_para_lev)
    MSE_para_fast_list[size_num] = np.mean(MSE_para_fast)
    MSE_para_OLS_list[size_num] = np.mean(MSE_para_OLS)
    MSE_para_random_list[size_num] = np.mean(MSE_para_random)

    MSE_lev_std_list[size_num] = np.std(MSE_lev)
    MSE_fast_std_list[size_num] = np.std(MSE_fast)
    MSE_OLS_std_list[size_num] = np.std(MSE_OLS)
    MSE_random_std_list[size_num] = np.std(MSE_random)
    MSE_para_lev_std_list[size_num] = np.std(MSE_para_lev)
    MSE_para_fast_std_list[size_num] = np.std(MSE_para_fast)
    MSE_para_OLS_std_list[size_num] = np.std(MSE_para_OLS)
    MSE_para_random_std_list[size_num] = np.std(MSE_para_random)

    time_lev_list[size_num] = np.mean(time_lev)
    time_fast_list[size_num] = np.mean(time_fast)
    time_OLS_list[size_num] = np.mean(time_OLS)
    time_random_list[size_num] = np.mean(time_random)

    time_lev_std_list[size_num] = np.std(time_lev)
    time_fast_std_list[size_num] = np.std(time_fast)
    time_OLS_std_list[size_num] = np.std(time_OLS)
    time_random_std_list[size_num] = np.std(time_random)

    size_num = size_num + 1

In [ ]:
plt.errorbar(subsample_size_list, MSE_para_lev_list, yerr = MSE_para_lev_std_list, color = 'orange', fmt = 'o-', label = 'LEV-exact')
plt.errorbar(subsample_size_list, MSE_para_fast_list, yerr = MSE_para_fast_std_list, color = 'red', fmt = '^-', label = 'LEV-appr')
plt.errorbar(subsample_size_list, MSE_para_OLS_list, yerr = MSE_para_OLS_std_list, color = 'black', fmt = 's-', label = 'OLS')
plt.errorbar(subsample_size_list, MSE_para_random_list, yerr = MSE_para_random_std_list, color = 'blue', fmt = '.-', label = 'UNIF')
plt.xscale('log')
plt.yscale('log')
plt.xticks(ticks=[1e2,3e2, 1e3, 3e3, 1e4], labels=['$10^2$', rf'$3\times 10^2$', '$10^3$', rf'$3\times 10^3$', '$10^4$'])
plt.xlabel('subsample size s', fontsize = 20)
plt.ylabel('MSE', fontsize = 20)
plt.tick_params(axis='both', labelsize=14)
plt.legend(fontsize = 12)